In [47]:
import sys
sys.path.append("D:/ML_Study/2017-CCF-BDCI-AIJudge")

import codecs
from collections import namedtuple

import numpy as np
import pandas as pd
from gensim.models import Doc2Vec
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from utils import LOGGER
from config.db_config import Config
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
config = Config()

## 加载标签数据

In [36]:
data_df = pd.read_csv(config.data_csv_path, encoding='utf-8')
y = data_df['penalty'] - 1
rows = data_df.shape[0]
num_class = len(np.unique(y))
print(rows,num_class)
data_df.head()

100 8


,id,content,laws,penalty
0,16,公诉 机关 霍邱县 人民检察院 被告人 许某 1975 日生 2012 因涉嫌 危险 驾驶 ...,"133,72,73",3
1,32,公诉 机关 海口市 龙华区 人民检察院 被告人 王某 海口市 龙华区 人民检察院 海龙 检公...,"347,67,52,64",1
2,41,公诉 机关 广东省 潮州市 人民检察院 被告人 覃学彬 1980 出生 广西壮族自治区 大新...,"263,25,52,53,55,56",5
3,57,公诉 机关 榆林市 榆阳区 人民检察院 上诉人 原审 被告人 2012 因涉嫌 盗窃罪 榆林...,"264,52,53,67,72,73",5
4,60,公诉 机关 榆阳区 人民检察院 上诉人 原审 被告人 刘某 汉族 陕西省 横山县 小学文化 ...,"224,25,26,27,52,72",7


,id,content,laws,penalty
0,16,公诉 机关 霍邱县 人民检察院 被告人 许某 1975 日生 2012 因涉嫌 危险 驾驶 ...,"133,72,73",3
1,32,公诉 机关 海口市 龙华区 人民检察院 被告人 王某 海口市 龙华区 人民检察院 海龙 检公...,"347,67,52,64",1
2,41,公诉 机关 广东省 潮州市 人民检察院 被告人 覃学彬 1980 出生 广西壮族自治区 大新...,"263,25,52,53,55,56",5
3,57,公诉 机关 榆林市 榆阳区 人民检察院 上诉人 原审 被告人 2012 因涉嫌 盗窃罪 榆林...,"264,52,53,67,72,73",5
4,60,公诉 机关 榆阳区 人民检察院 上诉人 原审 被告人 刘某 汉族 陕西省 横山县 小学文化 ...,"224,25,26,27,52,72",7


# 加载pre-train模型

In [37]:
# 模型加载-初始化文档的特征
pre_train_model = Doc2Vec.load(config.model_d2v_dm)
X = np.array([pre_train_model.docvecs[idx] for idx in range(rows)])

## 切分数据集/初始化数据矩阵向量

In [ ]:
# 切分数据： 训练集，测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print("dataset size X_train={0},X_test={1},dim={2}".format(X_train.shape[0], X_test.shape[0], X_test.shape[1]))
# 初始化数据矩阵，默认数据为0
stack = np.zeros((X_train.shape[0], num_class))
stack_te = np.zeros((X_test.shape[0], num_class))

In [44]:
stack.shape,stack_te.shape

((70, 8), (30, 8))

## 交叉验证- 特征得分平均

In [50]:
k = 5
print("cross_validate k={0}".format(k))
score_va = 0
score_te = 0
skf = StratifiedKFold(n_splits=k, random_state=config.seed)
for i, (train_index, val_index) in enumerate(skf.split(X_train, y_train)):
    print("cross_validate.....i={0}".format(i + 1))
    train_data, val_data = X_train[train_index], X_train[val_index]
    train_y, val_y = y_train[train_index], y_train[val_index]
    

cross_validate k=5
cross_validate.....i=1
cross_validate.....i=2
cross_validate.....i=3
cross_validate.....i=4
cross_validate.....i=5


D:\opt\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
D:\opt\anaconda3\lib\site-packages\pandas\core\series.py:841: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]
